# 521153S Deep Learning Final Project

In [1]:
# import necessary packages
import os
import requests
import zipfile
import sys
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.io import read_image
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, ResNet18_Weights, ResNet34_Weights, ResNet50_Weights, ResNet101_Weights, ResNet152_Weights, vgg11, vgg13, vgg16, vgg19, VGG11_Weights, VGG13_Weights, VGG16_Weights, VGG19_Weights
import gdown
import urllib
import os
import random
import tarfile
import time
import shutil
import logging as lg
from tqdm.notebook import tqdm
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

In [2]:
# Set the logging level
# Possible Levels: DEBUG, INFO,
lg.basicConfig(level=lg.DEBUG)

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# If you encounter some issues regarding cuda device, e.g., "RuntimeError: CUDA Out of memory error",
# try to switch the device to cpu by using the following code

# device = torch.device('cpu')
print('Device:', device)

Device: cuda:0


In [4]:
# Image Size
image_size = 224

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers (id not good, decrease learning rate (try 0.001, 0.0001, 0.00001))
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2

In [5]:
# download the dataset
val_url = 'https://drive.google.com/u/0/uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl'
train_url = 'https://drive.google.com/u/0/uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs'
test_url = 'https://drive.google.com/u/0/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v'
eurosat_url = 'https://zenodo.org/records/7711810/files/EuroSAT_RGB.zip?download=1'
cub_url = 'https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1'

val_file = './data/val.tar'
train_file = './data/train.tar'
test_file = './data/test.tar'
eurosat_file = './data/eurosatrgb.zip'
cub_file = './data/cub_200_2011.tgz'


if not os.path.exists('./data'):
    print('Creating data directory')
    os.mkdir('./data')

if not os.path.exists('./data/val.tar'):
    print('Downloading val.tar')
    gdown.download(val_url, val_file)
    val_tar = tarfile.open(val_file)
    val_tar.extractall('./data/')
    val_tar.close()

if not os.path.exists(train_file):
    print('Downloading train.tar')
    gdown.download(train_url, train_file)
    train_tar = tarfile.open(train_file)
    train_tar.extractall('./data/')
    train_tar.close()

if not os.path.exists(test_file):
    print('Downloading test.tar')
    gdown.download(test_url, test_file)
    test_tar = tarfile.open(test_file)
    test_tar.extractall('./data/')
    test_tar.close()

if not os.path.exists(eurosat_file):
    print('Downloading EuroSAT_RGB.zip')
    response = urllib.request.urlretrieve(eurosat_url, eurosat_file)
    eurosat_zip = zipfile.ZipFile(eurosat_file)
    eurosat_zip.extractall('./data/eurosat')
    eurosat_zip.close()

if not os.path.exists(cub_file):
    print('Downloading cub200_2011.tgz')
    response = urllib.request.urlretrieve(cub_url, cub_file)
    tar = tarfile.open(cub_file, 'r')
    for item in tar:
        tar.extract(item, './data')
    tar.close()


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): drive.google.com:443


Creating data directory


DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /u/0/uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl HTTP/1.1" 303 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): doc-04-6o-docs.googleusercontent.com:443
DEBUG:urllib3.connectionpool:https://doc-04-6o-docs.googleusercontent.com:443 "GET /docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/s0jtnmhu824ud1pk4tv4m63dv9j78368/1703166525000/12865399289486813135/*/1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl?uuid=788df922-2bda-4a14-9eb7-26ccff04bd7a HTTP/1.1" 200 30791680
Downloading...
From: https://drive.google.com/u/0/uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl
To: d:\koodi\Deep-Learning-Project\data\val.tar
100%|██████████| 30.8M/30.8M [00:01<00:00, 18.9MB/s]
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): drive.google.com:443


DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /u/0/uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs&confirm=t&uuid=57c652d0-28c4-45e9-a89a-a636a845eb5a HTTP/1.1" 303 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): doc-0c-6o-docs.googleusercontent.com:443
DEBUG:urllib3.connectionpool:https://doc-0c-6o-docs.googleusercontent.com:443 "GET /docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/80eu5d9oibjg5teuhkiq2dcm67la05p2/1703166525000/12865399289486813135/*/107FTosYIeBn5QbynR46YG91nHcJ70whs?uuid=57c652d0-28c4-45e9-a89a-a636a845eb5a HTTP/1.1" 200 125900800
Downloading...
From (uriginal): https://drive.google.com/u/0/uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs
From (redirected): https://drive.google.com/uc?id=107FTosYIeBn5QbynR46YG91

DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /u/0/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v&confirm=t&uuid=9df87679-206c-4f02-8c82-f1f43dbce9e3 HTTP/1.1" 303 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): doc-10-6o-docs.googleusercontent.com:443
DEBUG:urllib3.connectionpool:https://doc-10-6o-docs.googleusercontent.com:443 "GET /docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q6ofjkm9fcq69cvf6ps5mv5chqqv5blm/1703166675000/12865399289486813135/*/1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v?uuid=9df87679-206c-4f02-8c82-f1f43dbce9e3 HTTP/1.1" 200 39178240
Downloading...
From (uriginal): https://drive.google.com/u/0/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v
From (redirected): https://drive.google.com/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpH

In [6]:
model_resnet18 = resnet18(weights=ResNet18_Weights.DEFAULT)
model_resnet34 = resnet34(weights=ResNet34_Weights.DEFAULT)
model_resnet50 = resnet50(weights=ResNet50_Weights.DEFAULT)
model_resnet101 = resnet101(weights=ResNet101_Weights.DEFAULT)
model_resnet152 = resnet152(weights=ResNet152_Weights.DEFAULT)

model_vgg11 = vgg11(weights=VGG11_Weights.DEFAULT)
model_vgg13 = vgg13(weights=VGG13_Weights.DEFAULT)
model_vgg16 = vgg16(weights=VGG16_Weights.DEFAULT)
model_vgg19 = vgg19(weights=VGG19_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\joona/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 35.0MB/s]
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\joona/.cache\torch\hub\checkpoints\resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:02<00:00, 36.5MB/s]
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\joona/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 36.4MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to C:\Users\joona/.cache\torch\hub\checkpoints\resnet101-cd907fc2.pth
100%|██████████| 171M/171M [00:04<00:00, 35.9MB/s] 
Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to C:\Users\joona/.cache\torch\hub\checkpoints\resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:06<00:00, 36.6MB/s] 
Downloading: "https://d

In [ ]:
"""
train_dataset = torchvision.datasets.ImageFolder(
    root='./data/train',
    transform=transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
    ])
)


test_dataset = torchvision.datasets.ImageFolder(
    root='./data/test',
    transform=transforms.Compose([
        #transforms.Resize(image_size),
        #transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
)

validation_dataset = torchvision.datasets.ImageFolder(
    root='./data/val',
    transform=transforms.Compose([
        #transforms.Resize(image_size),
        #transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
)


dataloader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
dataloader_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
dataloader_validation = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
"""

In [7]:
# Stratified split, tries to maintain a proportional distribution of samples for each class in all three sets (train, val and test).

# Val and Test ratio total, Train ratio (1-test_ratio)
test_ratio = 0.3

# Split between Val and Test
test_val_split = 0.5

# Loading the ImageFolder dataset
train_dataset = torchvision.datasets.ImageFolder(root='./data/train')

# Getting the number of classes
num_classes = len(train_dataset.classes)

# Initializing lists to store indices for train, validation, and test sets
train_indices, val_indices, test_indices = [], [], []

# Splitting each class separately
for class_index in range(num_classes):
    # Getting indices for samples belonging to the current class
    class_indices = np.where(np.array(train_dataset.targets) == class_index)[0]
    
    # Splitting indices for each class separately into train, validation, and test sets
    train_idx, temp_idx = train_test_split(class_indices, test_size=test_ratio, random_state=42)
    val_idx, test_idx = train_test_split(temp_idx, test_size=test_val_split, random_state=42)
    
    # Extending the lists to store indices for each set
    train_indices.extend(train_idx)
    val_indices.extend(val_idx)
    test_indices.extend(test_idx)

# Creating Subset objects using the selected indices for each set
train_set = Subset(train_dataset, train_indices)
val_set = Subset(train_dataset, val_indices)
test_set = Subset(train_dataset, test_indices)

# Transform functions
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(image_size),      
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

transform_eval = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

train_set.dataset.transform = transform_train
val_set.dataset.transform = transform_eval
test_set.dataset.transform = transform_eval

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Printing the sizes of the resulting sets
print(f"Number of samples in the training set: {len(train_set)}")
print(f"Number of samples in the validation set: {len(val_set)}")
print(f"Number of samples in the test set: {len(test_set)}")

Number of samples in the training set: 26880
Number of samples in the validation set: 5760
Number of samples in the test set: 5760


In [8]:
# Define the used model
model = model_resnet18
model.name = "resnet18"     # Set name for saving

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [9]:
def evaluate(model, data_loader, device, criterion):

    model.eval() 
    # valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    num_images = 0.0 # used to accumulate number of images
    
    #with torch.no_grad():
    for i, data in enumerate(data_loader, 0):
        counter += 1
        
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        # Forward pass.
        outputs = model(image)
        # Calculate the loss.
        loss = criterion(outputs, labels)
        # valid_running_loss += loss.item()
        # Calculate the accuracy.
        #preds = outputs.argmax(dim=1)
        preds = torch.max(outputs, 1).indices
        
        #valid_running_correct += preds.eq(labels).sum()
        valid_running_correct += torch.sum(preds == labels.data)
        num_images += len(labels)
        
    # Loss and accuracy for the complete epoch.
    # epoch_loss = valid_running_loss / counter
    lg.debug(f"EVAL:   loss.item(): {loss.item()} valid_running_correct: {valid_running_correct.item()}")

    epoch_acc = valid_running_correct / num_images
    return epoch_acc

In [11]:
def train(model, trainLoader, valLoader, criterion, optimizer, num_epochs, device):
    model.to(device)
    train_loss = []
    train_acc = []

    # Start the training.
    if torch.cuda.is_available():
        print("Using CUDA")

    print('Training')
    
    lg.debug(f" len(trainLoader.dataset): {len(trainLoader.dataset)}")
    lg.debug(f" len(valLoader.dataset): {len(valLoader.dataset)}")

    
    for epoch in range(num_epochs):
        model.train()
        train_running_loss = 0.0
        train_running_correct = 0
        counter = 0
        num_images = 0.0 # used to accumulate number of images
        for i, data in enumerate(trainLoader,0):
            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            train_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            train_running_correct += torch.sum(preds == labels.data)
            # Backpropagation
            loss.backward()
            # Update the weights.
            optimizer.step()    # Output training 
            num_images += len(image)
            if counter % 100 == 0:
                print(f'Epoch: {epoch+1}/{num_epochs}, Step: {counter}/{len(trainLoader)}, Loss: {loss.item():.4f}')
        
        # reduce learning rate every 10 epochs by factor of 10
        if (epoch+1) % 10 == 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 10.0


        # Validation
        valid_epoch_acc = evaluate(model, valLoader, device, criterion)
        lg.debug(f"TRAIN: loss.item(): {loss.item()}, train_running_correct: {train_running_correct}")
        
        # train_epoch_loss = train_running_loss / counter
        # train_loss.append(train_epoch_loss)
        # valid_loss.append(valid_epoch_loss)
        train_epoch_acc = train_running_correct  / num_images
        # train_acc.append(train_epoch_acc )

        print('Epoch: %d/%d, Training accuracy: %f, Loss: %f, Validation accuracy: %f' % (epoch+1, num_epochs, train_epoch_acc, loss.item(), valid_epoch_acc))

        # valid_acc.append(valid_epoch_acc)
        # lg.debug(f"Training loss mean: {train_epoch_loss :.3f}")
        # print(f"Validation acc: {valid_epoch_acc:.3f}")
        print('-'*50)


    return model

In [ ]:
# pretrained_model = train(model, dataloader_train, dataloader_validation, criterion, optimizer, num_epochs, device)

In [12]:
pretrained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

DEBUG:root: len(trainLoader.dataset): 26880
DEBUG:root: len(valLoader.dataset): 5760


Using CUDA
Training
Epoch: 1/10, Step: 100/210, Loss: 2.8659
Epoch: 1/10, Step: 200/210, Loss: 1.6891


DEBUG:root:EVAL:   loss.item(): 2.3503589630126953 valid_running_correct: 4052
DEBUG:root:TRAIN: loss.item(): 1.6037564277648926, train_running_correct: 11122


Epoch: 1/10, Training accuracy: 0.413765, Loss: 1.603756, Validation accuracy: 0.703472
--------------------------------------------------
Epoch: 2/10, Step: 100/210, Loss: 1.1886
Epoch: 2/10, Step: 200/210, Loss: 1.0192


DEBUG:root:EVAL:   loss.item(): 1.3323090076446533 valid_running_correct: 4499
DEBUG:root:TRAIN: loss.item(): 0.9654290080070496, train_running_correct: 20247


...

Epoch: 8/10, Training accuracy: 0.934226, Loss: 0.385586, Validation accuracy: 0.845139
--------------------------------------------------
Epoch: 9/10, Step: 100/210, Loss: 0.2560
Epoch: 9/10, Step: 200/210, Loss: 0.2963


DEBUG:root:EVAL:   loss.item(): 0.7581312656402588 valid_running_correct: 4884
DEBUG:root:TRAIN: loss.item(): 0.2746165096759796, train_running_correct: 25450


Epoch: 9/10, Training accuracy: 0.946801, Loss: 0.274617, Validation accuracy: 0.847917
--------------------------------------------------
Epoch: 10/10, Step: 100/210, Loss: 0.2354
Epoch: 10/10, Step: 200/210, Loss: 0.2001


DEBUG:root:EVAL:   loss.item(): 0.7032350301742554 valid_running_correct: 4907
DEBUG:root:TRAIN: loss.item(): 0.2885981500148773, train_running_correct: 25765


Epoch: 10/10, Training accuracy: 0.958519, Loss: 0.288598, Validation accuracy: 0.851910
--------------------------------------------------


In [13]:
def save_model(trained_model):
    # Create Models Folder
    if not os.path.exists('./models'):
        print('Creating models directory')
        os.mkdir('./models')

    # Save the model
    if not os.path.exists(f'./models/{trained_model.name}.pth'):
        print(f'Saving Model {trained_model.name}')
        torch.save(trained_model, f'./models/{trained_model.name}.pth')

def load_model(model_name):
    if not os.path.exists(f'./models/{model_name}.pth'):
        print(f'Cannot Load {model_name}')
        return None
    print(f'Loading Model {model_name}')
    model = torch.load(f'./models/{model_name}.pth')
    model.eval()
    return model

In [14]:
save_model(pretrained_model)

In [15]:
# validate_loss, validate_acc = evaluate(trained_model, dataloader_validation, criterion, device)
# print(f"Validation loss: {validate_loss:.3f}, validation acc: {validate_acc:.3f}")
test_acc = evaluate(pretrained_model, test_loader, device, criterion)
print(f"test acc: {test_acc:.3f}")

DEBUG:root:EVAL:   loss.item(): 0.8170550465583801 valid_running_correct: 4870


test acc: 0.845


In [16]:
def choose_100_eurosat():
    '''
    The `choose_100_eurosat()` function selects 100 images from the EuroSAT dataset and copies them to the `data/eurosat_validation` folder. It also creates a `data/eurosat_training` folder if it doesn't already exist.
    '''
    if not os.path.exists('./data/eurosat_validation'):
        print('Creating data directory')
        os.mkdir('./data/eurosat_validation')
    else:
        # Folder full of images -> delete all files in the directory
        shutil.rmtree(os.path.join('./data/eurosat_validation'))
        os.mkdir('./data/eurosat_validation')

    if not os.path.exists('./data/eurosat_training'):
        print('Creating training directory')
        os.mkdir('./data/eurosat_training')
    else:
        # Folder full of images -> delete all files in the directory
        shutil.rmtree(os.path.join('./data/eurosat_training'))
        os.mkdir('./data/eurosat_training')

    # Load the EuroSAT Categories
    eurosat_categories = [name for name in os.listdir('./data/eurosat/EuroSAT_RGB/') if os.path.isdir(os.path.join('./data/eurosat/EuroSAT_RGB/', name))]
    # Randomly select 5 categories
    lg.debug(f"Selecting 5 categories from {eurosat_categories} categories")
    selected_categories = np.random.choice(eurosat_categories, 5, replace=False)
    lg.debug(f"Selected categories: {selected_categories}")

    selected_images = []
    training_images = []

    # From each directory, randomly select 20 images
    for category in selected_categories:
        images = os.listdir(os.path.join('./data/eurosat/EuroSAT_RGB/', category))
        selected = random.sample(images, 20)
        selected_images.extend([(category, image) for image in selected])
        lg.debug(f"Selected {selected} from {category}")
        # Copy selected images to the selected directory
        for image in selected:
            if not os.path.exists(f"./data/eurosat_validation/{category}"):
                lg.debug(f"Creating {category} directory")
                os.mkdir(f"./data/eurosat_validation/{category}")
            shutil.copyfile(os.path.join('./data/eurosat/EuroSAT_RGB', category, image), os.path.join(f"./data/eurosat_validation/{category}", image))

    # From these 100 images, randomly select 5 images from each category for the training set
    for category in selected_categories:
        category_images = [image for (cat, image) in selected_images if cat == category]
        training = random.sample(category_images, 5)
        training_images.extend(training)
        lg.debug(f"Selected {training} for training")

        # Copy training images to the training directory
        for image in training:
            if not os.path.exists(f"./data/eurosat_training/{category}"):
                lg.debug(f"Creating {category} directory")
                os.mkdir(f"./data/eurosat_training/{category}")
            shutil.move(os.path.join(f"./data/eurosat_validation/{category}", image), os.path.join(f"./data/eurosat_training/{category}", image))

In [17]:
batch_size = 12

In [18]:
def load_euro_datasets():

    # Choose 100 images from EuroSAT dataset
    choose_100_eurosat()

    eurosat_train_dataset = torchvision.datasets.ImageFolder(
        root='./data/eurosat_training',
        transform=transforms.Compose([
            transforms.RandomResizedCrop(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
        ])
    )

    eurosat_validation_dataset = torchvision.datasets.ImageFolder(
        root='./data/eurosat_validation',
        transform=transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    )

    eurosat_dataloader_train = DataLoader(eurosat_train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    eurosat_dataloader_validation = DataLoader(eurosat_validation_dataset, batch_size=batch_size, shuffle = False, num_workers=num_workers)

    return eurosat_dataloader_train, eurosat_dataloader_validation

In [19]:
# Load the pretrained model
model_resnet18_fine = load_model("resnet18")

for param in model_resnet18_fine.parameters():
    param.requires_grad = False

model_resnet18_fine.fc = nn.Linear(model_resnet18_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet18_fine.parameters(), lr=0.001, momentum=0.9)

num_epochs = 30

# Load the EuroSAT dataset
eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()

if os.path.exists("results.txt"):
    # If the file exists, clear its contents
    with open("results.txt", "w") as f:
        f.truncate(0)  # Clear the file content
        print("File 'results.txt' found and cleared.")

# Train and Evaluate the model on the EuroSAT dataset
i = 5
average_results = 0.0

for j in range(i):
    print(f"Training EuroSAT model {j+1}/{i}")
    eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()
    model_resnet18_fine = train(model_resnet18_fine, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)
    results = evaluate(model_resnet18_fine, eurosat_dataloader_validation, device, criterion)
    average_results += results
    print(f"EuroSAT model {j+1}/{i} results: {results:.3f}")
    # Save the results to results.txt
    with open("results.txt", "a") as f:
        f.write(f"{model_resnet18_fine.name} {j} Time: {time.strftime('%d/%m/%Y, %H:%M:%S')} Epochs: {num_epochs} | Results: {results:.6f}\n")
        # f.close()

average_results /= i
print(f"Average results: {average_results:.3f}")
with open("results.txt", "a") as f:
        f.write(f"{model_resnet18_fine.name} Average results: {average_results:.6f}\n")
        # f.close()

DEBUG:root:Selecting 5 categories from ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake'] categories
DEBUG:root:Selected categories: ['AnnualCrop' 'HerbaceousVegetation' 'River' 'Pasture' 'SeaLake']
DEBUG:root:Selected ['AnnualCrop_1605.jpg', 'AnnualCrop_1053.jpg', 'AnnualCrop_660.jpg', 'AnnualCrop_1947.jpg', 'AnnualCrop_1641.jpg', 'AnnualCrop_1819.jpg', 'AnnualCrop_1496.jpg', 'AnnualCrop_1078.jpg', 'AnnualCrop_2852.jpg', 'AnnualCrop_752.jpg', 'AnnualCrop_1607.jpg', 'AnnualCrop_923.jpg', 'AnnualCrop_1423.jpg', 'AnnualCrop_1763.jpg', 'AnnualCrop_42.jpg', 'AnnualCrop_72.jpg', 'AnnualCrop_2376.jpg', 'AnnualCrop_1154.jpg', 'AnnualCrop_2098.jpg', 'AnnualCrop_711.jpg'] from AnnualCrop
DEBUG:root:Creating AnnualCrop directory


Loading Model resnet18
Creating data directory
Creating training directory


DEBUG:root:Selected ['HerbaceousVegetation_2155.jpg', 'HerbaceousVegetation_1073.jpg', 'HerbaceousVegetation_1248.jpg', 'HerbaceousVegetation_2167.jpg', 'HerbaceousVegetation_2884.jpg', 'HerbaceousVegetation_2454.jpg', 'HerbaceousVegetation_1998.jpg', 'HerbaceousVegetation_2582.jpg', 'HerbaceousVegetation_2528.jpg', 'HerbaceousVegetation_2567.jpg', 'HerbaceousVegetation_2104.jpg', 'HerbaceousVegetation_2685.jpg', 'HerbaceousVegetation_2548.jpg', 'HerbaceousVegetation_2618.jpg', 'HerbaceousVegetation_910.jpg', 'HerbaceousVegetation_1051.jpg', 'HerbaceousVegetation_1726.jpg', 'HerbaceousVegetation_1812.jpg', 'HerbaceousVegetation_1292.jpg', 'HerbaceousVegetation_2906.jpg'] from HerbaceousVegetation
DEBUG:root:Creating HerbaceousVegetation directory
DEBUG:root:Selected ['River_29.jpg', 'River_2173.jpg', 'River_1335.jpg', 'River_825.jpg', 'River_2479.jpg', 'River_1030.jpg', 'River_33.jpg', 'River_2232.jpg', 'River_1369.jpg', 'River_1653.jpg', 'River_2331.jpg', 'River_1140.jpg', 'River_1618

File 'results.txt' found and cleared.
Training EuroSAT model 1/5


DEBUG:root:Selected ['Residential_10.jpg', 'Residential_1467.jpg', 'Residential_257.jpg', 'Residential_1501.jpg', 'Residential_1265.jpg', 'Residential_2119.jpg', 'Residential_474.jpg', 'Residential_273.jpg', 'Residential_86.jpg', 'Residential_851.jpg', 'Residential_1402.jpg', 'Residential_300.jpg', 'Residential_1364.jpg', 'Residential_64.jpg', 'Residential_389.jpg', 'Residential_1293.jpg', 'Residential_2744.jpg', 'Residential_1295.jpg', 'Residential_166.jpg', 'Residential_263.jpg'] from Residential
DEBUG:root:Creating Residential directory
DEBUG:root:Selected ['HerbaceousVegetation_993.jpg', 'HerbaceousVegetation_1960.jpg', 'HerbaceousVegetation_2774.jpg', 'HerbaceousVegetation_55.jpg', 'HerbaceousVegetation_2288.jpg'] for training
DEBUG:root:Creating HerbaceousVegetation directory
DEBUG:root:Selected ['AnnualCrop_1648.jpg', 'AnnualCrop_1874.jpg', 'AnnualCrop_1352.jpg', 'AnnualCrop_7.jpg', 'AnnualCrop_355.jpg'] for training
DEBUG:root:Creating AnnualCrop directory
DEBUG:root:Selected [

...

Epoch: 4/30, Training accuracy: 0.280000, Loss: 1.602229, Validation accuracy: 0.333333
--------------------------------------------------


DEBUG:root:EVAL:   loss.item(): 0.5640082359313965 valid_running_correct: 18
DEBUG:root:TRAIN: loss.item(): 0.7296133041381836, train_running_correct: 12


Epoch: 5/30, Training accuracy: 0.480000, Loss: 0.729613, Validation accuracy: 0.240000
--------------------------------------------------


DEBUG:root:EVAL:   loss.item(): 0.5316546559333801 valid_running_correct: 20
DEBUG:root:TRAIN: loss.item(): 1.9472284317016602, train_running_correct: 7


Epoch: 6/30, Training accuracy: 0.280000, Loss: 1.947228, Validation accuracy: 0.266667
--------------------------------------------------


In [ ]:
# Calculate saved EuroSAT results average
total_acc = 0
count = 0

# Open the file and read lines
with open("results.txt", "r") as file:
    lines = file.readlines()

    # Iterate through each line
    for line in lines:
        # Split the line to extract the accuracy value (assuming it's the 7th element when splitting by space)
        line_parts = line.split()
        if len(line_parts) >= 7:
            results_acc = float(line_parts[6])  # Assuming accuracy is the 7th element
            total_acc += results_acc
            count += 1

# Calculate the average accuracy
if count > 0:
    average_acc = total_acc / count
    print(f"Average results accuracy: {average_acc:.6f}")
else:
    print("No accuracy values found in the file or the file is empty.")

## RESNET 34

In [ ]:
# Image Size
image_size = 224

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers (id not good, decrease learning rate (try 0.001, 0.0001, 0.00001))
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2

In [ ]:
# Define the used model
model = model_resnet34
model.name = "resnet34"     # Set name for saving

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
pretrained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

In [ ]:
save_model(pretrained_model)

In [ ]:
# validate_loss, validate_acc = evaluate(trained_model, dataloader_validation, criterion, device)
# print(f"Validation loss: {validate_loss:.3f}, validation acc: {validate_acc:.3f}")
test_acc = evaluate(pretrained_model, test_loader, device, criterion)
print(f"test acc: {test_acc:.3f}")

In [ ]:
batch_size = 12

In [ ]:
# Load the pretrained model
model_resnet34_fine = load_model("resnet34")

for param in model_resnet34_fine.parameters():
    param.requires_grad = False

model_resnet34_fine.fc = nn.Linear(model_resnet34_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet34_fine.parameters(), lr=0.001, momentum=0.9)

num_epochs = 30

# Load the EuroSAT dataset
eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()

eurosat_trained_model = train(model_resnet34_fine, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)


In [ ]:
i = 11

for j in range(i):
    print(f"Training EuroSAT model {j+1}/{i}")
    eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()
    eurosat_trained_model = train(eurosat_trained_model, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)
    results = evaluate(eurosat_trained_model, eurosat_dataloader_validation, device, criterion)
    print(f"EuroSAT model {j+1}/{i} results: {results:.3f}")
    # Save the results to results.txt
    with open("results.txt", "a") as f:
        f.write(f"{eurosat_trained_model.name} {j} Time: {time.strftime('%d/%m/%Y, %H:%M:%S')} Epochs: {num_epochs} | Results: {results:.6f}\n")
        f.close()

## RESNET 50

In [ ]:
# Image Size
image_size = 224

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers (id not good, decrease learning rate (try 0.001, 0.0001, 0.00001))
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2


# Define the used model
model = model_resnet50
model.name = "resnet50"     # Set name for saving

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

pretrained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

In [ ]:
save_model(pretrained_model)

# validate_loss, validate_acc = evaluate(trained_model, dataloader_validation, criterion, device)
# print(f"Validation loss: {validate_loss:.3f}, validation acc: {validate_acc:.3f}")
test_acc = evaluate(pretrained_model, test_loader, device, criterion)
print(f"test acc: {test_acc:.3f}")

In [ ]:
batch_size = 12


# Load the pretrained model
model_resnet50_fine = load_model("resnet50")

for param in model_resnet50_fine.parameters():
    param.requires_grad = False

model_resnet50_fine.fc = nn.Linear(model_resnet50_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet50_fine.parameters(), lr=0.001, momentum=0.9)

num_epochs = 30

# Load the EuroSAT dataset
eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()

eurosat_trained_model = train(model_resnet50_fine, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)

In [ ]:
i = 11
j = 1

for j in range(i):
    print(f"Training EuroSAT model {j+1}/{i}")
    eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()
    eurosat_trained_model = train(eurosat_trained_model, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)
    results = evaluate(eurosat_trained_model, eurosat_dataloader_validation, device, criterion)
    print(f"EuroSAT model {j+1}/{i} results: {results:.3f}")
    # Save the results to results.txt
    with open("results.txt", "a") as f:
        f.write(f"{j} Time: {time.strftime('%d/%m/%Y, %H:%M:%S')} Epochs: {num_epochs} | Results: {results:.6f}\n")
        f.close()

## DATASET: CUB

In [42]:
batch_size = 16

In [51]:
cub_dataset = torchvision.datasets.ImageFolder(root='./data/CUB_200_2011/images')

# Transform functions
transform_eval = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

cub_dataset.transform = transform_eval

# Loading the ImageFolder dataset


# Getting the number of classes
cub_num_classes = len(cub_dataset.classes)

cub_val_loader = DataLoader(cub_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

# Printing the sizes of the resulting sets
print(f"Number of samples in the validation set: {len(cub_dataset)}")


Number of samples in the validation set: 11788


In [52]:
# Batch size during training
num_workers = 1

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2


# Load the pretrained model
model_resnet18_fine = load_model("resnet18")

for param in model_resnet18_fine.parameters():
    param.requires_grad = False


model_resnet18_fine.fc = nn.Linear(model_resnet18_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet18_fine.parameters(), lr=0.001, momentum=0.9)

model_resnet18_fine.to(device)

if torch.cuda.is_available():
    print("Using CUDA")

cub_model = evaluate(model_resnet18_fine, cub_val_loader, device, criterion)

Loading Model resnet18


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
